In [209]:
import pandas as pd
%matplotlib inline

In [210]:
#loading the data
pump_data = pd.read_csv(r'Training_set_values.csv')
labels = pd.read_csv(r'Training_set_labels.csv')


In [211]:
#initial look at the data
display(pump_data.head(5))
display(labels.head(5))


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [212]:
#merge the data into one dataframe
pump_df = pd.merge(pump_data, labels, on='id')
print(pump_df.shape)

prediction_columns = []


(59400, 41)


### Checking null values in the dataframe


In [213]:
#check null values
nulls = (pump_df.isnull().sum(axis=0))
print(nulls)

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [214]:
nan_col = [i for i in pump_df.columns if pump_df[i].isnull().any()]
print(f'Columns containing null values are: {nan_col}')

Columns containing null values are: ['funder', 'installer', 'subvillage', 'public_meeting', 'scheme_management', 'scheme_name', 'permit']


In [215]:
#function for changing null values with the above columns

def check_change_null_values(df, col, no_top ):
    top = df[col].value_counts().nlargest(no_top).index.tolist()
    print(f"The top {[col]}, from {df[col].nunique()} unique {col} are:\n{top}")

    df.loc[~df[col].isin(top), [col]] = 'Other'
    print(f'\nThe resulting classifications are:\n{df[col].value_counts()}')

#### Funding

In [216]:
#use top 8 funders and classify the rest as other
check_change_null_values(pump_df, 'funder', 8)

prediction_columns.append('funder')

The top ['funder'], from 1897 unique funder are:
['Government Of Tanzania', 'Danida', 'Hesawa', 'Rwssp', 'World Bank', 'Kkkt', 'World Vision', 'Unicef']

The resulting classifications are:
Other                     38687
Government Of Tanzania     9084
Danida                     3114
Hesawa                     2202
Rwssp                      1374
World Bank                 1349
Kkkt                       1287
World Vision               1246
Unicef                     1057
Name: funder, dtype: int64


#### Installation


In [217]:
print(pump_df.installer.nunique())
print(pump_df.installer.value_counts())
#Could use top 8 again (in line with funders) however the 8th value is '0' which represents an unknown. Therefore, installers falling outside the top 7 will be classified as 'Other'

2145
DWE                17402
Government          1825
RWE                 1206
Commu               1060
DANIDA              1050
                   ...  
Wizara  ya maji        1
TWESS                  1
Nasan workers          1
R                      1
SELEPTA                1
Name: installer, Length: 2145, dtype: int64


In [218]:
check_change_null_values(pump_df, 'installer', 7)

prediction_columns.append('installer')


The top ['installer'], from 2145 unique installer are:
['DWE', 'Government', 'RWE', 'Commu', 'DANIDA', 'KKKT', 'Hesawa']

The resulting classifications are:
Other         35119
DWE           17402
Government     1825
RWE            1206
Commu          1060
DANIDA         1050
KKKT            898
Hesawa          840
Name: installer, dtype: int64


#### Sub village

In [219]:
print(pump_df.subvillage.value_counts())
print(pump_df.subvillage.nunique())
print(pump_df.subvillage.isnull().sum())

Madukani        508
Shuleni         506
Majengo         502
Kati            373
Mtakuja         262
               ... 
Kipompo           1
Chanyamilima      1
Ikalime           1
Kemagaka          1
Kikatanyemba      1
Name: subvillage, Length: 19287, dtype: int64
19287
371


In [220]:
print(f'There are {pump_df.subvillage.nunique()} unique sub-villages listed within the dataframe.  As no one sub-village dominates the values, this column will be ignored in the predictions - as there will be too many variables to work with.')

There are 19287 unique sub-villages listed within the dataframe.  As no one sub-village dominates the values, this column will be ignored in the predictions - as there will be too many variables to work with.


#### Public meeting

In [221]:
print(pump_df.public_meeting.value_counts())
print(pump_df.public_meeting.nunique())
print(pump_df.public_meeting.isnull().sum())

True     51011
False     5055
Name: public_meeting, dtype: int64
2
3334


In [222]:
#This variable will need more investigation to see if it is worth including in the predictions. For now, the null values will be changed to "Unknown"

pump_df.public_meeting.fillna('Unknown', inplace=True)
print(pump_df.public_meeting.isnull().sum())
print(pump_df.public_meeting)

investigate_columns = []
investigate_columns.append('public_meeting')

0
0           True
1        Unknown
2           True
3           True
4           True
          ...   
59395       True
59396       True
59397       True
59398       True
59399       True
Name: public_meeting, Length: 59400, dtype: object


#### Scheme management

In [223]:
print(pump_df.scheme_management.value_counts())
print(pump_df.scheme_management.nunique())
print(pump_df.scheme_management.isnull().sum())

VWC                 36793
WUG                  5206
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64
12
3877


In [224]:
#There are 12 unique variables. Any below "Other" (the top 8) will be combined into "Other"
check_change_null_values(pump_df, 'scheme_management', 8)

The top ['scheme_management'], from 12 unique scheme_management are:
['VWC', 'WUG', 'Water authority', 'WUA', 'Water Board', 'Parastatal', 'Private operator', 'Company']

The resulting classifications are:
VWC                 36793
WUG                  5206
Other                4813
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Name: scheme_management, dtype: int64


In [225]:
prediction_columns.append('scheme_management')

#### Scheme name

In [226]:
print(pump_df.scheme_name.value_counts())
print(pump_df.scheme_name.nunique())
print(pump_df.scheme_name.isnull().sum())

K                       682
None                    644
Borehole                546
Chalinze wate           405
M                       400
                       ... 
Mradi wa maji Vijini      1
Villagers                 1
Magundi water supply      1
Saadani Chumv             1
Mtawanya                  1
Name: scheme_name, Length: 2696, dtype: int64
2696
28166


In [227]:
print(f'There are {pump_df.scheme_name.nunique()} unique schemes listed within the dataframe along with {pump_df.scheme_name.isnull().sum()} null values.  As no one scheme dominates the values and there are many unknowns, this column will be ignored in the predictions - as there will be too many variables to work with.')

There are 2696 unique schemes listed within the dataframe along with 28166 null values.  As no one scheme dominates the values and there are many unknowns, this column will be ignored in the predictions - as there will be too many variables to work with.


#### Permit

In [228]:
print(pump_df.permit.value_counts())
print(pump_df.permit.nunique())
print(pump_df.permit.isnull().sum())

#this column can be treated in a similar manner to the public meeting column

True     38852
False    17492
Name: permit, dtype: int64
2
3056


In [229]:
pump_df.permit.fillna('Unknown', inplace=True)
print(pump_df.permit.isnull().sum())
print(pump_df.permit)

investigate_columns.append('permit')

0
0        False
1         True
2         True
3         True
4         True
         ...  
59395     True
59396     True
59397    False
59398     True
59399     True
Name: permit, Length: 59400, dtype: object


## Columns with string values

In [230]:
print(pump_df.dtypes)

id                         int64
amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_ty

In [231]:
string_col = [i for i in pump_df.columns if pump_df[i].dtypes == 'object']
print(string_col)

['date_recorded', 'funder', 'installer', 'wpt_name', 'basin', 'subvillage', 'region', 'lga', 'ward', 'public_meeting', 'recorded_by', 'scheme_management', 'scheme_name', 'permit', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'quantity_group', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group', 'status_group']


In [232]:
string_col_sort = set(string_col) - set(nan_col)
print(string_col_sort)
print(len(string_col_sort))
print(len(nan_col))
print(len(string_col))

{'recorded_by', 'extraction_type', 'status_group', 'management', 'management_group', 'source_type', 'quantity_group', 'source', 'extraction_type_group', 'source_class', 'quality_group', 'waterpoint_type_group', 'date_recorded', 'ward', 'region', 'basin', 'water_quality', 'payment_type', 'waterpoint_type', 'wpt_name', 'quantity', 'payment', 'extraction_type_class', 'lga'}
24
7
31


#### Extraction type / extraction type class / extraction type group


In [233]:
#the kind of extraction the water pump uses
print(pump_df.extraction_type.value_counts())
print(pump_df.extraction_type_group.value_counts())
print(pump_df.extraction_type_class.value_counts())
print(pump_df.extraction_type.nunique())
print(pump_df.extraction_type_group.nunique())
print(pump_df.extraction_type_class.nunique())

gravity                      26780
nira/tanira                   8154
other                         6430
submersible                   4764
swn 80                        3670
mono                          2865
india mark ii                 2400
afridev                       1770
ksb                           1415
other - rope pump              451
other - swn 81                 229
windmill                       117
india mark iii                  98
cemo                            90
other - play pump               85
walimi                          48
climax                          32
other - mkulima/shinyanga        2
Name: extraction_type, dtype: int64
gravity            26780
nira/tanira         8154
other               6430
submersible         6179
swn 80              3670
mono                2865
india mark ii       2400
afridev             1770
rope pump            451
other handpump       364
other motorpump      122
wind-powered         117
india mark iii        98
Name: ext

In [234]:
print('These columns all contain the same information, but appears to have been further sorted which iteration. Therefore is seems sensible to use the extraction_type_group column in predictions to reduce the number of variables.')

These columns all contain the same information, but appears to have been further sorted which iteration. Therefore is seems sensible to use the extraction_type_group column in predictions to reduce the number of variables.


In [235]:
prediction_columns.append('extraction_type_group')

#### Management

In [236]:
pump_df.filter(regex=('manage'))

,scheme_management,management,management_group
0,VWC,vwc,user-group
1,Other,wug,user-group
2,VWC,vwc,user-group
3,VWC,vwc,user-group
4,Other,other,other
...,...,...,...
59395,Water Board,water board,user-group
59396,VWC,vwc,user-group
59397,VWC,vwc,user-group
59398,VWC,vwc,user-group


In [237]:
print(pump_df.management.value_counts())
print(pump_df.management_group.value_counts())
print(pump_df.scheme_management.value_counts())
print(pump_df.management.nunique())
print(pump_df.management_group.nunique())
print(pump_df.scheme_management.nunique())


vwc                 40507
wug                  6515
water board          2933
wua                  2535
private operator     1971
parastatal           1768
water authority       904
other                 844
company               685
unknown               561
other - school         99
trust                  78
Name: management, dtype: int64
user-group    52490
commercial     3638
parastatal     1768
other           943
unknown         561
Name: management_group, dtype: int64
VWC                 36793
WUG                  5206
Other                4813
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Name: scheme_management, dtype: int64
12
5
9


In [238]:
print('The same information is contained within the columns management, management_group and scheme_management. Therefore only scheme_management will be used within the prediction.')

The same information is contained within the columns management, management_group and scheme_management. Therefore only scheme_management will be used within the prediction.


In [239]:
prediction_columns.append('scheme_management')

#### Water quality

In [240]:
print(pump_df.water_quality.value_counts())
print(pump_df.quality_group.value_counts())
print(pump_df.water_quality.nunique())
print(pump_df.quality_group.nunique())

soft                  50818
salty                  4856
unknown                1876
milky                   804
coloured                490
salty abandoned         339
fluoride                200
fluoride abandoned       17
Name: water_quality, dtype: int64
good        50818
salty        5195
unknown      1876
milky         804
colored       490
fluoride      217
Name: quality_group, dtype: int64
8
6


In [241]:
print('The water_quality and quality_group columns contain the same information. The quality_group column will be used for predictions as is does not segregate abandoned pumps - which could affect predictions.  The abandoned pumps could be used in a later prediction model, to see if there is any effect on prediction success.')

The water_quality and quality_group columns contain the same information. The quality_group column will be used for predictions as is does not segregate abandoned pumps - which could affect predictions.  The abandoned pumps could be used in a later prediction model, to see if there is any effect on prediction success.


In [242]:
prediction_columns.append('quality_group')

#### Source


In [243]:
pump_df.filter(regex=('source'))

,source,source_type,source_class
0,spring,spring,groundwater
1,rainwater harvesting,rainwater harvesting,surface
2,dam,dam,surface
3,machine dbh,borehole,groundwater
4,rainwater harvesting,rainwater harvesting,surface
...,...,...,...
59395,spring,spring,groundwater
59396,river,river/lake,surface
59397,machine dbh,borehole,groundwater
59398,shallow well,shallow well,groundwater


In [244]:
print(pump_df.source.value_counts())
print(pump_df.source_type.value_counts())
print(pump_df.source_class.value_counts())
print(pump_df.source.nunique())
print(pump_df.source_type.nunique())
print(pump_df.source_class.nunique())


spring                  17021
shallow well            16824
machine dbh             11075
river                    9612
rainwater harvesting     2295
hand dtw                  874
lake                      765
dam                       656
other                     212
unknown                    66
Name: source, dtype: int64
spring                  17021
shallow well            16824
borehole                11949
river/lake              10377
rainwater harvesting     2295
dam                       656
other                     278
Name: source_type, dtype: int64
groundwater    45794
surface        13328
unknown          278
Name: source_class, dtype: int64
10
7
3


In [245]:
print('The source columns have the same information, but further grouped with each iteration. To use a good selection of data (i.e. not too many variables and not too much data loss through grouping) the source_type column will be used in predictions.')

The source columns have the same information, but further grouped with each iteration. To use a good selection of data (i.e. not too many variables and not too much data loss through grouping) the source_type column will be used in predictions.


In [246]:
prediction_columns.append('source_type')

#### Quantity

In [247]:
pump_df.filter(regex=('quant'))

,quantity,quantity_group
0,enough,enough
1,insufficient,insufficient
2,enough,enough
3,dry,dry
4,seasonal,seasonal
...,...,...
59395,enough,enough
59396,enough,enough
59397,enough,enough
59398,insufficient,insufficient


In [248]:
print(pump_df.quantity.value_counts())
print(pump_df.quantity_group.value_counts())
print(pump_df.quantity.nunique())
print(pump_df.quantity_group.nunique())

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity, dtype: int64
enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity_group, dtype: int64
5
5


In [249]:
print('These two columns are duplicates. Therefore only quantity will be used.')
prediction_columns.append('quantity')

These two columns are duplicates. Therefore only quantity will be used.


#### Payment

In [250]:
print(pump_df.payment.value_counts())
print(pump_df.payment_type.value_counts())
print(pump_df.payment.nunique())
print(pump_df.payment_type.nunique())

never pay                25348
pay per bucket            8985
pay monthly               8300
unknown                   8157
pay when scheme fails     3914
pay annually              3642
other                     1054
Name: payment, dtype: int64
never pay     25348
per bucket     8985
monthly        8300
unknown        8157
on failure     3914
annually       3642
other          1054
Name: payment_type, dtype: int64
7
7


In [251]:
print('These two columns are duplicates. Therefore only payment will be used.')
prediction_columns.append('payment')

These two columns are duplicates. Therefore only payment will be used.


#basin, region, ward, lga
#### Geographic

In [252]:
print(pump_df.basin.value_counts())
print(pump_df.ward.value_counts())
print(pump_df.basin.nunique())
print(pump_df.ward.nunique())

Lake Victoria              10248
Pangani                     8940
Rufiji                      7976
Internal                    7785
Lake Tanganyika             6432
Wami / Ruvu                 5987
Lake Nyasa                  5085
Ruvuma / Southern Coast     4493
Lake Rukwa                  2454
Name: basin, dtype: int64
Igosi              307
Imalinyi           252
Siha Kati          232
Mdandu             231
Nduruma            217
                  ... 
Uchindile            1
Thawi                1
Uwanja wa Ndege      1
Izia                 1
Kinungu              1
Name: ward, Length: 2092, dtype: int64
9
2092


In [253]:
print(pump_df.region.value_counts())
print(pump_df.lga.value_counts())
print(pump_df.region.nunique())
print(pump_df.lga.nunique())

Iringa           5294
Shinyanga        4982
Mbeya            4639
Kilimanjaro      4379
Morogoro         4006
Arusha           3350
Kagera           3316
Mwanza           3102
Kigoma           2816
Ruvuma           2640
Pwani            2635
Tanga            2547
Dodoma           2201
Singida          2093
Mara             1969
Tabora           1959
Rukwa            1808
Mtwara           1730
Manyara          1583
Lindi            1546
Dar es Salaam     805
Name: region, dtype: int64
Njombe          2503
Arusha Rural    1252
Moshi Rural     1251
Bariadi         1177
Rungwe          1106
                ... 
Moshi Urban       79
Kigoma Urban      71
Arusha Urban      63
Lindi Urban       21
Nyamagana          1
Name: lga, Length: 125, dtype: int64
21
125


In [254]:
#Region, ward, lga and basin are all geographical information about the pumps. This data is likely to be highly correlated so one column will need to be chosen. The numeric columns district_code and region_code contain geographical information, so the columns to use within the prediction model will be decided when the numeric columns have been inspected.

In [255]:
print(pump_df.region_code.value_counts())
print(pump_df.region_code.nunique())
#region codes do not provide any further information, therefore will be excluded from the initial prediction model.

11    5300
17    5011
12    4639
3     4379
5     4040
18    3324
19    3047
2     3024
16    2816
10    2640
4     2513
1     2201
13    2093
14    1979
20    1969
15    1808
6     1609
21    1583
80    1238
60    1025
90     917
7      805
99     423
9      390
24     326
8      300
40       1
Name: region_code, dtype: int64
27


In [256]:
#Will use the basin column as it has a reasonable number of variables and groups the pumps into areas.  Other columns could be considered in other prediction model iterations
prediction_columns.append('basin')

In [257]:
print(pump_df.waterpoint_type.value_counts())
print(pump_df.waterpoint_type_group.value_counts())
print(pump_df.waterpoint_type.nunique())
print(pump_df.waterpoint_type_group.nunique())

communal standpipe             28522
hand pump                      17488
other                           6380
communal standpipe multiple     6103
improved spring                  784
cattle trough                    116
dam                                7
Name: waterpoint_type, dtype: int64
communal standpipe    34625
hand pump             17488
other                  6380
improved spring         784
cattle trough           116
dam                       7
Name: waterpoint_type_group, dtype: int64
7
6


In [258]:
print('The waterpoint columns contain duplicate information. The waterpoint_type column contains a further category where there are multiple communal standpipes. As this may affect the running/repairs of a pump, waterpoint_type will be used in prediction.')
prediction_columns.append('waterpoint_type')

The waterpoint columns contain duplicate information. The waterpoint_type column contains a further category where there are multiple communal standpipes. As this may affect the running/repairs of a pump, waterpoint_type will be used in prediction.


In [259]:
print(pump_df.recorded_by.value_counts())
#This column only has one company - the consultants that recorded the data - therefore it will not be used in the prediction model.

GeoData Consultants Ltd    59400
Name: recorded_by, dtype: int64


## Numerical columns

#### Amount of water


In [260]:
print(pump_df.amount_tsh.value_counts())
print(pump_df.amount_tsh.nunique())
#Large number of variables and large percent of null data (represented by 0s), therefore this will not be used within the prediction model

0.0         41639
500.0        3102
50.0         2472
1000.0       1488
20.0         1463
            ...  
6300.0          1
120000.0        1
138000.0        1
350000.0        1
59.0            1
Name: amount_tsh, Length: 98, dtype: int64
98


#### Num_private

In [261]:
print(pump_df.num_private.value_counts())
#There is no information about the data this column contains and as there are a large number of variables, it will be excluded from the prediction model.

0       58643
6          81
1          73
5          46
8          46
        ...  
42          1
23          1
136         1
698         1
1402        1
Name: num_private, Length: 65, dtype: int64


#### Population

In [262]:
print(pump_df.population.value_counts())
print(pump_df.population.nunique())

#there is a lot of information here, and a large number of variables/null values (zeros). These could be further explored, but will be removed from the initial prediction model.

0       21381
1        7025
200      1940
150      1892
250      1681
        ...  
6330        1
5030        1
656         1
948         1
788         1
Name: population, Length: 1049, dtype: int64
1049


#### GPS, latitude and longitude

In [263]:
print(pump_df.gps_height.value_counts())
print(pump_df.latitude.value_counts())
print(pump_df.longitude.value_counts())

#too many variables as each pump will have a different geographical location

 0       20438
-15         60
-16         55
-13         55
 1290       52
         ...  
 2378        1
-54          1
 2057        1
 2332        1
 2366        1
Name: gps_height, Length: 2428, dtype: int64
-2.000000e-08    1812
-6.985842e+00       2
-6.980220e+00       2
-2.476680e+00       2
-6.978263e+00       2
                 ... 
-3.287619e+00       1
-8.234989e+00       1
-3.268579e+00       1
-1.146053e+01       1
-6.747464e+00       1
Name: latitude, Length: 57517, dtype: int64
0.000000     1812
37.375717       2
38.340501       2
39.086183       2
33.005032       2
             ... 
35.885754       1
36.626541       1
37.333530       1
38.970078       1
38.104048       1
Name: longitude, Length: 57516, dtype: int64


#### Construction and age

In [264]:
print(pump_df.construction_year.value_counts())
print(pump_df.construction_year.nunique())
#There are 55 different years listed.

0       20709
2010     2645
2008     2613
2009     2533
2000     2091
2007     1587
2006     1471
2003     1286
2011     1256
2004     1123
2012     1084
2002     1075
1978     1037
1995     1014
2005     1011
1999      979
1998      966
1990      954
1985      945
1980      811
1996      811
1984      779
1982      744
1994      738
1972      708
1974      676
1997      644
1992      640
1993      608
2001      540
1988      521
1983      488
1975      437
1986      434
1976      414
1970      411
1991      324
1989      316
1987      302
1981      238
1977      202
1979      192
1973      184
2013      176
1971      145
1960      102
1967       88
1963       85
1968       77
1969       59
1964       40
1962       30
1961       21
1965       19
1966       17
Name: construction_year, dtype: int64
55


In [265]:
print(pump_df.date_recorded.max())
print(pump_df.date_recorded.min())
#The dates recorded span a decade. The age of the pump is likely to affect whether a pump is functional or in need of repair. Therefore creation of a new column seems like the best solution.

print(pump_df.construction_year.max())
print(pump_df.construction_year.min())

#Will run into problems with the null (zero values). Therefore will use a median value to fill in these.


2013-12-03
2002-10-14
2013
0


In [266]:
#first convert the date_recorded column to datetime format as it is currently a string.
pump_df.date_recorded = pd.to_datetime(pump_df.date_recorded)
print(pump_df.date_recorded.dtype)


datetime64[ns]


In [267]:
pump_df['year_recorded'] = pump_df.date_recorded.dt.year
print(pump_df.year_recorded)

0        2011
1        2013
2        2013
3        2013
4        2011
         ... 
59395    2013
59396    2011
59397    2011
59398    2011
59399    2011
Name: year_recorded, Length: 59400, dtype: int64


In [268]:
print(pump_df.year_recorded.median())
print(pump_df.year_recorded.mean())
print(pump_df.construction_year.median())
print(pump_df.construction_year.mean())

2012.0
2011.9216666666666
1986.0
1300.6524747474748


In [269]:
pump_df.loc[pump_df['year_recorded'] == 0, 'year_recorded'] = 2012
print(pump_df.year_recorded.value_counts())

2011    28674
2013    24271
2012     6424
2004       30
2002        1
Name: year_recorded, dtype: int64


In [270]:
pump_df.loc[pump_df['construction_year'] == 0, 'construction_year'] = 1986
print(pump_df.construction_year.value_counts())

1986    21143
2010     2645
2008     2613
2009     2533
2000     2091
2007     1587
2006     1471
2003     1286
2011     1256
2004     1123
2012     1084
2002     1075
1978     1037
1995     1014
2005     1011
1999      979
1998      966
1990      954
1985      945
1996      811
1980      811
1984      779
1982      744
1994      738
1972      708
1974      676
1997      644
1992      640
1993      608
2001      540
1988      521
1983      488
1975      437
1976      414
1970      411
1991      324
1989      316
1987      302
1981      238
1977      202
1979      192
1973      184
2013      176
1971      145
1960      102
1967       88
1963       85
1968       77
1969       59
1964       40
1962       30
1961       21
1965       19
1966       17
Name: construction_year, dtype: int64


In [271]:
pump_df['age_at_record'] = pump_df.year_recorded - pump_df.construction_year
print(pump_df.age_at_record)
print(pump_df.age_at_record.value_counts())


0        12
1         3
2         4
3        27
4        25
         ..
59395    14
59396    15
59397    25
59398    25
59399     9
Name: age_at_record, Length: 59400, dtype: int64
 25    13926
 26     5731
 27     2972
 3      2740
 1      2303
 2      2129
 5      1980
 4      1890
 13     1869
 7      1404
 6      1381
 11     1352
 8      1160
 14     1160
 33     1120
 15      993
 23      905
 10      868
 16      827
 9       814
 18      767
 19      766
 31      709
 28      709
 35      647
 17      636
 39      601
 37      600
 0       588
 21      578
 29      564
 12      559
 20      541
 41      488
 30      359
 43      356
 38      351
 36      325
 22      323
 24      261
 40      233
 34      184
 32      174
 53       91
 42       90
 50       84
 45       66
 46       48
 44       47
 48       41
 51       31
 49       25
 47       14
 52       11
-5         3
-4         2
-2         1
-7         1
-1         1
-3         1
Name: age_at_record, dtype: int64


In [272]:
#remove negative values
pump_df.loc[pump_df.age_at_record < 0, 'age_at_record'] = 0
print(pump_df.age_at_record.value_counts())

25    13926
26     5731
27     2972
3      2740
1      2303
2      2129
5      1980
4      1890
13     1869
7      1404
6      1381
11     1352
8      1160
14     1160
33     1120
15      993
23      905
10      868
16      827
9       814
18      767
19      766
31      709
28      709
35      647
17      636
39      601
37      600
0       597
21      578
29      564
12      559
20      541
41      488
30      359
43      356
38      351
36      325
22      323
24      261
40      233
34      184
32      174
53       91
42       90
50       84
45       66
46       48
44       47
48       41
51       31
49       25
47       14
52       11
Name: age_at_record, dtype: int64


In [273]:
prediction_columns.append('age_at_record')

In [276]:
print(prediction_columns)
print(investigate_columns)
prediction_columns.append('id')
prediction_columns.append('status_group')

['funder', 'installer', 'scheme_management', 'extraction_type_group', 'scheme_management', 'quality_group', 'source_type', 'quantity', 'payment', 'basin', 'waterpoint_type', 'age_at_record']
['public_meeting', 'permit']


In [280]:
#The columns to be investigated further will be added to the new dataframe for use in data analysis before applying the prediction model.

model_df_list = prediction_columns + investigate_columns
print(model_df_list)

['funder', 'installer', 'scheme_management', 'extraction_type_group', 'scheme_management', 'quality_group', 'source_type', 'quantity', 'payment', 'basin', 'waterpoint_type', 'age_at_record', 'id', 'status_group', 'public_meeting', 'permit']


In [288]:
model_df = pump_df[model_df_list].copy()
print(model_df)

           funder installer scheme_management extraction_type_group  \
0           Other     Other               VWC               gravity   
1           Other     Other             Other               gravity   
2           Other     Other               VWC               gravity   
3          Unicef     Other               VWC           submersible   
4           Other     Other             Other               gravity   
...           ...       ...               ...                   ...   
59395       Other     Other       Water Board               gravity   
59396       Other     Other               VWC               gravity   
59397       Other     Other               VWC                swn 80   
59398       Other     Other               VWC           nira/tanira   
59399  World Bank     Other               VWC           nira/tanira   

      scheme_management quality_group           source_type      quantity  \
0                   VWC          good                spring        eno

In [290]:
model_df.to_csv('pump_model_data.csv', index=False)